## Predicting US Elections Using Sentiment Analysis

In [28]:
# Importing essential libraries
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

from textblob import TextBlob
from wordcloud import WordCloud
import plotly.graph_objects as go 
import plotly.express as px

In [29]:
# Importing reviews
trump_reviews= pd.read_csv('Trumpall2.csv')
biden_reviews= pd.read_csv("Bidenall2.csv")

In [30]:
# Getting a glimpse of the data
trump_reviews.head(), biden_reviews.head()

(              user                                               text
 0      manny_rosen   @sanofi please tell us how many shares the Cr...
 1        osi_abdul   https://t.co/atM98CpqF7  Like, comment, RT #P...
 2          Patsyrw   Your AG Barr is as useless &amp; corrupt as y...
 3  seyedebrahimi_m   Mr. Trump! Wake Up!  Most of the comments bel...
 4    James09254677   After 4 years you think you would have figure...,
            user                                               text
 0   MarkHodder3    @JoeBiden And we’ll find out who won in 2026...
 1    K87327961G  @JoeBiden Your Democratic Nazi Party cannot be...
 2      OldlaceA                        @JoeBiden So did Lying Barr
 3    penblogger  @JoeBiden It's clear you didnt compose this tw...
 4  Aquarian0264         @JoeBiden I will vote in person thank you.)

### Sentiment Analysis

In [31]:
''' 
Using TextBlob library to perform text classification in either positive or negative
'''
textblob1= TextBlob(trump_reviews["text"][10])
print("Trump :", textblob1.sentiment)

textblob2= TextBlob(biden_reviews["text"][500])
print("Biden :", textblob2.sentiment)

Trump : Sentiment(polarity=0.15, subjectivity=0.3125)
Biden : Sentiment(polarity=0.6, subjectivity=0.9)


In [32]:
'''
Polarity ranges from -1 to +1(negative to positive) and tells whether the text has negative sentiments or positive sentiments. Polarity tells about factual information.
'''
def find_pol(review):
    return TextBlob(review).sentiment.polarity
trump_reviews["Sentiment Polarity"]= trump_reviews["text"].apply(find_pol)
print(trump_reviews.tail())

biden_reviews["Sentiment Polarity"]= biden_reviews["text"].apply(find_pol)
print(biden_reviews.tail())

                 user                                               text  \
2783          4diva63  @realDonaldTrump For the 1/100 time, absentee ...   
2784         hidge826  @realDonaldTrump If you’re so scared of losing...   
2785     SpencerRossy  @realDonaldTrump I rarely get involved with fo...   
2786  ScoobyMcpherson  @realDonaldTrump This is the moment when Trump...   
2787          bjklinz     @realDonaldTrump I’m sorry, Donald. No. #POTUS   

      Sentiment Polarity  
2783               0.000  
2784               0.000  
2785               0.225  
2786               0.000  
2787              -0.500  
             user                                               text  \
2535    meryn1977  @JoeBiden You'll just try to calm those waters...   
2536  BSNelson114  @JoeBiden 96 days 96 dias #VoteJoeBiden2020  #...   
2537     KenCapel  @JoeBiden YOU THINK YOU CAN DO THAT??? YOU CAN...   
2538   LeslyeHale  @JoeBiden Trump wants our children back at sch...   
2539     rerickre  @J

In [33]:
''' Adding a new attribute Expression Label that will classify the text as positive or negative on the basis of polarity'''
trump_reviews['Expression_Label']= np.where(trump_reviews['Sentiment Polarity']>0, 'positive', 'negative')
trump_reviews["Expression_Label"][trump_reviews["Sentiment Polarity"]==0]="Neutral"
print(trump_reviews.tail())


biden_reviews['Expression_Label']= np.where(biden_reviews['Sentiment Polarity']>0, 'positive', 'negative')
biden_reviews["Expression_Label"][biden_reviews["Sentiment Polarity"]==0]="Neutral"
print(biden_reviews.tail())

                 user                                               text  \
2783          4diva63  @realDonaldTrump For the 1/100 time, absentee ...   
2784         hidge826  @realDonaldTrump If you’re so scared of losing...   
2785     SpencerRossy  @realDonaldTrump I rarely get involved with fo...   
2786  ScoobyMcpherson  @realDonaldTrump This is the moment when Trump...   
2787          bjklinz     @realDonaldTrump I’m sorry, Donald. No. #POTUS   

      Sentiment Polarity Expression_Label  
2783               0.000          Neutral  
2784               0.000          Neutral  
2785               0.225         positive  
2786               0.000          Neutral  
2787              -0.500         negative  
             user                                               text  \
2535    meryn1977  @JoeBiden You'll just try to calm those waters...   
2536  BSNelson114  @JoeBiden 96 days 96 dias #VoteJoeBiden2020  #...   
2537     KenCapel  @JoeBiden YOU THINK YOU CAN DO THAT??? YOU C

<ipython-input-33-dcf258845511>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-33-dcf258845511>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [34]:
# Droping all the tweets marked with neutral polarity from both the datastes to balance the data.
reviews1= trump_reviews[trump_reviews['Sentiment Polarity']==0.00000]
print(reviews1.shape)

cond1=trump_reviews['Sentiment Polarity'].isin(reviews1['Sentiment Polarity'])
trump_reviews.drop(trump_reviews[cond1].index, inplace = True)
print(trump_reviews.shape)


reviews2= biden_reviews[biden_reviews['Sentiment Polarity']==0.00000]
print(reviews2.shape)

cond2=biden_reviews['Sentiment Polarity'].isin(reviews2['Sentiment Polarity'])
biden_reviews.drop(biden_reviews[cond2].index, inplace = True)
print(biden_reviews.shape)

(1464, 4)
(1324, 4)
(1509, 4)
(1031, 4)


In [35]:
# Balancing the dataset
# Donald Trump
np.random.seed(10)
remove_n =324
drop_indices = np.random.choice(trump_reviews.index, remove_n, replace=False)
df_subset_trump = trump_reviews.drop(drop_indices)
print(df_subset_trump.shape)
# Joe Biden
np.random.seed(10)
remove_n =31
drop_indices = np.random.choice(biden_reviews.index, remove_n, replace=False)
df_subset_biden = biden_reviews.drop(drop_indices)
print(df_subset_biden.shape)

(1000, 4)
(1000, 4)


In [38]:
#### Analysing the data for positive and negative sentiments
count_1 = df_subset_trump.groupby('Expression_Label').count()
print(count_1)

negative_per1 = (count_1['Sentiment Polarity'][0]/1000)*10
positive_per1 = (count_1['Sentiment Polarity'][1]/1000)*100

count_2 = df_subset_biden.groupby('Expression_Label').count()
print(count_2)

negative_per2 = (count_2['Sentiment Polarity'][0]/1000)*100
positive_per2 = (count_2['Sentiment Polarity'][1]/1000)*100

Politicians = ['Joe Biden', 'Donald Trump']
lis_pos = [positive_per1, positive_per2]
lis_neg = [negative_per1, negative_per2]

fig = go.Figure(data=[
    go.Bar(name='Positive', x=Politicians, y=lis_pos),
    go.Bar(name='Negative', x=Politicians, y=lis_neg)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

                  user  text  Sentiment Polarity
Expression_Label                                
negative           449   449                 449
positive           551   551                 551
                  user  text  Sentiment Polarity
Expression_Label                                
negative           393   393                 393
positive           607   607                 607
